In [10]:
from datetime import datetime
import pandas as pd
import yaml
from sqlalchemy import create_engine

## Database Connection

In [11]:
with open('config.yml', 'r') as f: #Abrir el archivo en modo de  lectura
    config = yaml.safe_load(f) # Crear un diccionario con lo que hay en el archivo
    config_db_etl = config['bodega'] #Obtener solo la configuración de la bodega
    config_db = config["fuente"] #Obtener solo la configuración de la bodega

In [12]:
# Construct the database URL
url_db_etl = (f"{config_db_etl['driver']}://{config_db_etl['user']}:{config_db_etl['password']}@{config_db_etl['host']}:"
           f"{config_db_etl['port']}/{config_db_etl['db']}")
url_db = (f"{config_db['driver']}://{config_db['user']}:{config_db['password']}@{config_db['host']}:"
           f"{config_db['port']}/{config_db['db']}")

In [13]:
# Create the SQLAlchemy Engine
etl_conn = create_engine(url_db_etl)
olap_conn = create_engine(url_db)

## Load table mensajero

In [14]:
mensajero = pd.read_sql_table("clientes_mensajeroaquitoy",url_db)
auth_user = pd.read_sql_table("auth_user",url_db)
ciudad = pd.read_sql_table("ciudad",url_db)
departamento = pd.read_sql_table("departamento",url_db)

In [15]:
mensajero.describe(include="all")
auth_user.describe(include="all")
ciudad.describe(include="all")
departamento.describe(include="all")

,departamento_id,nombre
count,4.000000,4
unique,NaN,4
top,NaN,NARIÑO
freq,NaN,1
mean,2.500000,NaN
std,1.290994,NaN
min,1.000000,NaN
25%,1.750000,NaN
50%,2.500000,NaN
75%,3.250000,NaN


In [16]:
mensajero.drop(columns=["fecha_entrada","fecha_salida","telefono","token_Firebase","url_foto"], inplace=True)
mensajero

,id,user_id,activo,salario,ciudad_operacion_id
0,1,2,True,NaN,13.0
1,42,410,True,NaN,1.0
2,48,447,True,1300000.0,1.0
3,41,400,True,NaN,1.0
4,13,331,True,1160000.0,4.0
5,28,346,True,116000.0,1.0
6,33,364,True,1160000.0,7.0
7,36,368,True,1300000.0,1.0
8,8,327,True,1160000.0,2.0
9,21,338,True,1160000.0,1.0


In [17]:
mensajero = mensajero.merge(
    ciudad,
    left_on='ciudad_operacion_id',  # Columna en el DataFrame de mensajeros
    right_on='ciudad_id',                   # Columna en el DataFrame de ciudades
    how='left'                       # Mantiene todos los registros del DataFrame de mensajeros
)
mensajero

,id,user_id,activo,salario,ciudad_operacion_id,ciudad_id,nombre,departamento_id
0,1,2,True,NaN,13.0,13.0,ACOPI YUMBO,1.0
1,42,410,True,NaN,1.0,1.0,CALI,1.0
2,48,447,True,1300000.0,1.0,1.0,CALI,1.0
3,41,400,True,NaN,1.0,1.0,CALI,1.0
4,13,331,True,1160000.0,4.0,4.0,PASTO,4.0
5,28,346,True,116000.0,1.0,1.0,CALI,1.0
6,33,364,True,1160000.0,7.0,7.0,TULUA,1.0
7,36,368,True,1300000.0,1.0,1.0,CALI,1.0
8,8,327,True,1160000.0,2.0,2.0,PALMIRA,1.0
9,21,338,True,1160000.0,1.0,1.0,CALI,1.0


In [18]:
mensajero = mensajero.merge(
    departamento,
    left_on='departamento_id',  
    right_on='departamento_id',                   
    how='left'                       
)
mensajero

,id,user_id,activo,salario,ciudad_operacion_id,ciudad_id,nombre_x,departamento_id,nombre_y
0,1,2,True,NaN,13.0,13.0,ACOPI YUMBO,1.0,VALLE DEL CAUCA
1,42,410,True,NaN,1.0,1.0,CALI,1.0,VALLE DEL CAUCA
2,48,447,True,1300000.0,1.0,1.0,CALI,1.0,VALLE DEL CAUCA
3,41,400,True,NaN,1.0,1.0,CALI,1.0,VALLE DEL CAUCA
4,13,331,True,1160000.0,4.0,4.0,PASTO,4.0,NARIÑO
5,28,346,True,116000.0,1.0,1.0,CALI,1.0,VALLE DEL CAUCA
6,33,364,True,1160000.0,7.0,7.0,TULUA,1.0,VALLE DEL CAUCA
7,36,368,True,1300000.0,1.0,1.0,CALI,1.0,VALLE DEL CAUCA
8,8,327,True,1160000.0,2.0,2.0,PALMIRA,1.0,VALLE DEL CAUCA
9,21,338,True,1160000.0,1.0,1.0,CALI,1.0,VALLE DEL CAUCA


In [19]:
mensajero.rename(columns={"nombre_x": "ciudad_operacion", "nombre_y": "departamento_operacion"}, inplace=True)
mensajero.drop(columns=["ciudad_operacion_id","ciudad_id","departamento_id"], inplace=True)
mensajero


,id,user_id,activo,salario,ciudad_operacion,departamento_operacion
0,1,2,True,NaN,ACOPI YUMBO,VALLE DEL CAUCA
1,42,410,True,NaN,CALI,VALLE DEL CAUCA
2,48,447,True,1300000.0,CALI,VALLE DEL CAUCA
3,41,400,True,NaN,CALI,VALLE DEL CAUCA
4,13,331,True,1160000.0,PASTO,NARIÑO
5,28,346,True,116000.0,CALI,VALLE DEL CAUCA
6,33,364,True,1160000.0,TULUA,VALLE DEL CAUCA
7,36,368,True,1300000.0,CALI,VALLE DEL CAUCA
8,8,327,True,1160000.0,PALMIRA,VALLE DEL CAUCA
9,21,338,True,1160000.0,CALI,VALLE DEL CAUCA


In [20]:
auth_user.rename(columns={"id": "auth_user_id"}, inplace=True)
auth_user.drop(columns=["password","last_login","is_superuser","username","date_joined","is_active","is_staff"], inplace=True)
auth_user

,auth_user_id,first_name,last_name,email
0,318,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
1,322,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
2,326,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
3,327,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
4,328,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
...,...,...,...,...
324,498,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
325,493,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
326,500,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
327,504,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com


In [21]:
mensajero = mensajero.merge(
    auth_user,
    left_on='user_id',  
    right_on='auth_user_id',                   
    how='left'                       
)
mensajero

,id,user_id,activo,salario,ciudad_operacion,departamento_operacion,auth_user_id,first_name,last_name,email
0,1,2,True,NaN,ACOPI YUMBO,VALLE DEL CAUCA,2,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
1,42,410,True,NaN,CALI,VALLE DEL CAUCA,410,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
2,48,447,True,1300000.0,CALI,VALLE DEL CAUCA,447,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
3,41,400,True,NaN,CALI,VALLE DEL CAUCA,400,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
4,13,331,True,1160000.0,PASTO,NARIÑO,331,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
5,28,346,True,116000.0,CALI,VALLE DEL CAUCA,346,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
6,33,364,True,1160000.0,TULUA,VALLE DEL CAUCA,364,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
7,36,368,True,1300000.0,CALI,VALLE DEL CAUCA,368,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
8,8,327,True,1160000.0,PALMIRA,VALLE DEL CAUCA,327,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
9,21,338,True,1160000.0,CALI,VALLE DEL CAUCA,338,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com


In [22]:
mensajero.drop(columns=["user_id","auth_user_id"], inplace=True)
mensajero

,id,activo,salario,ciudad_operacion,departamento_operacion,first_name,last_name,email
0,1,True,NaN,ACOPI YUMBO,VALLE DEL CAUCA,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
1,42,True,NaN,CALI,VALLE DEL CAUCA,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
2,48,True,1300000.0,CALI,VALLE DEL CAUCA,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
3,41,True,NaN,CALI,VALLE DEL CAUCA,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
4,13,True,1160000.0,PASTO,NARIÑO,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
5,28,True,116000.0,CALI,VALLE DEL CAUCA,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
6,33,True,1160000.0,TULUA,VALLE DEL CAUCA,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
7,36,True,1300000.0,CALI,VALLE DEL CAUCA,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
8,8,True,1160000.0,PALMIRA,VALLE DEL CAUCA,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
9,21,True,1160000.0,CALI,VALLE DEL CAUCA,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com


In [23]:
mensajero.fillna("no_definido", inplace=True)
mensajero

C:\Users\nicol\AppData\Local\Temp\ipykernel_26208\1016313821.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'no_definido' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  mensajero.fillna("no_definido", inplace=True)


,id,activo,salario,ciudad_operacion,departamento_operacion,first_name,last_name,email
0,1,True,no_definido,ACOPI YUMBO,VALLE DEL CAUCA,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
1,42,True,no_definido,CALI,VALLE DEL CAUCA,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
2,48,True,1300000.0,CALI,VALLE DEL CAUCA,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
3,41,True,no_definido,CALI,VALLE DEL CAUCA,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
4,13,True,1160000.0,PASTO,NARIÑO,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
5,28,True,116000.0,CALI,VALLE DEL CAUCA,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
6,33,True,1160000.0,TULUA,VALLE DEL CAUCA,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
7,36,True,1300000.0,CALI,VALLE DEL CAUCA,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
8,8,True,1160000.0,PALMIRA,VALLE DEL CAUCA,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com
9,21,True,1160000.0,CALI,VALLE DEL CAUCA,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com


In [24]:
# Contar el número de valores nulos por columna
nulos_por_columna = mensajero.isnull().sum()
print("Número de valores nulos por columna:\n", nulos_por_columna)

Número de valores nulos por columna:
 id                        0
activo                    0
salario                   0
ciudad_operacion          0
departamento_operacion    0
first_name                0
last_name                 0
email                     0
dtype: int64


In [28]:
mensajero["saved"] = datetime.now()
mensajero.head()

,id,activo,salario,ciudad_operacion,departamento_operacion,first_name,last_name,email,saved
0,1,True,no_definido,ACOPI YUMBO,VALLE DEL CAUCA,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,2024-10-30 08:11:00.792891
1,42,True,no_definido,CALI,VALLE DEL CAUCA,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,2024-10-30 08:11:00.792891
2,48,True,1300000.0,CALI,VALLE DEL CAUCA,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,2024-10-30 08:11:00.792891
3,41,True,no_definido,CALI,VALLE DEL CAUCA,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,2024-10-30 08:11:00.792891
4,13,True,1160000.0,PASTO,NARIÑO,pepito_el_rapido,pepito_el_furioso,rapidos-furiosos@gmail.com,2024-10-30 08:11:00.792891


## Load table into ETL database

In [27]:
mensajero.to_sql("dim_mensajero", etl_conn, index_label="key_mensajero", if_exists="replace") 

50